# Lecture on Understanding High-Entropy RNGs with NIST benchmark 

    by M.Süzen
    (c) 2025
    
The power of `leymosun` package is it provides HE-RNG, i.e., non-deterministic seed at each call.
This generates high-entropy randomness compare to constant seed. Here we demonstrate that `leymosun` passes on average more tests comparatively with higher total score. This brings the advantage of having more "randomized" statistical properties, making `leymosun` good fit for numerical simulations with using non-deterministic seeds.

Our scoring function simply provides ratio of passed tests on NIST's [NIST.SP.800-22r1a]( https://doi.org/10.6028/NIST.SP.800-22r1a) test.

**Note**: We use `nistrng` package here. We need to install this notebook seperately, we don't install it by `leymosun` by default. One can install this via `!pip install nistrng` for example.  Also, replacing scoring method (`sp800`) with another python function that does the same scoring on the `NIST.SP.800-22r1a` tests is possible.

## Benchmark

We generate 10000 integers in the interval $[-128, 128]$. We observe roughly about on average `~4-7% higher passing scores` with the `1-3%` more test passes. HE-RNGs imporoved the quality of pass scores and test passes. This is computed via cumulative sums over 15 runs. See exercise for better estimation of uplifts. 

In [ ]:
import leymosun 
leymosun.__version__

In [ ]:
from nistrng import *
import numpy as np
from numpy.random import PCG64, RandomState

def run_sp800(integer_sequence):
    binary_sequence = pack_sequence(integer_sequence)
    eligible_battery = check_eligibility_all_battery(binary_sequence, SP800_22R1A_BATTERY)
    return run_all_battery(binary_sequence, eligible_battery, False)

def sp800score(integer_sequence):
    results = run_sp800(integer_sequence)
    total = 0.0
    test_names = []
    test_pass = []
    for result, _ in results:
        total =  result.score + total
        test_names.append(result.name)
        test_pass.append(result.passed)
    return test_names, test_pass, total  
    

In [ ]:
numpy_rng = RandomState(PCG64(seed=42))
frandint = numpy_rng.randint
grand_total = 0 
npass = 0
for _ in range(15):
    sequence = frandint(-128, 128, 10000, dtype=int)
    test_names, test_pass, total   = sp800score(sequence)
    grand_total = grand_total + total 
    npass = npass + np.sum(test_pass)
grand_total_m = grand_total
npass_m = npass
grand_total_m, npass_m

In [ ]:
from leymosun.random import randint as frandint
grand_total = 0 
npass = 0
for _ in range(15):
    sequence = frandint(-128, 128, 10000, dtype=int)
    test_names, test_pass, total   = sp800score(sequence)
    grand_total = grand_total + total 
    npass = npass + np.sum(test_pass)
grand_total, npass

In [ ]:
# Uplifts 
grand_total/grand_total_m, npass/npass_m

## Conclusion

We have demonstrated that using common seed (such as 42) compare to non-deterministic seeding degrates the quality of RNGs. 

**Exercise** Run the experiment above, calculate the cumulative scores uplift per test.

## References 

* [NIST.SP.800-22r1a]( https://doi.org/10.6028/NIST.SP.800-22r1a) A Statistical Test Suite for
Random and Pseudorandom Number Generators for Cryptographic Applications